In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('smsspamcollection.tsv', sep='\t')

In [4]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [5]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [6]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X = df['message']

In [9]:
y = df['label']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=42)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
count_vect = CountVectorizer()

In [15]:
# fit the vectorizer to the data (build a vocab, count the number of words...)
# count_vect.fit(X_train)
# X_train_counts = count_vect.transform(X_train)

# transform the original text message  -> VECTOR
X_train_counts = count_vect.fit_transform(X_train)

In [16]:
X_train_counts

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [17]:
X_train.shape

(3733,)

In [18]:
X_train_counts.shape

(3733, 7082)

In [19]:
from sklearn.feature_extraction.text import TfidfTransformer

In [20]:
tfidf_transformer = TfidfTransformer()

In [21]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [22]:
X_train_tfidf.shape

(3733, 7082)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
vactorizer = TfidfVectorizer()

In [31]:
X_train_tfidf = vactorizer.fit_transform(X_train)

In [32]:
# train a classifier
from sklearn.svm import LinearSVC

In [33]:
clf = LinearSVC()

In [34]:
clf.fit(X_train_tfidf, y_train)

LinearSVC()

In [35]:
from sklearn.pipeline import Pipeline

In [36]:
text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [37]:
text_clf.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [38]:
predictions = text_clf.predict(X_test)

In [39]:
from sklearn.metrics import confusion_matrix,classification_report

In [40]:
print(confusion_matrix(y_test, predictions))

[[1586    7]
 [  12  234]]


In [41]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [42]:
from sklearn import metrics

In [43]:
metrics.accuracy_score(y_test, predictions)

0.989668297988037

In [44]:
text_clf.predict(["hi how are you doing today?"])

array(['ham'], dtype=object)

In [45]:
text_clf.predict(["Congratulations, youve been selected as a vinner. TEXT WON 424242 congratulations free entry to contexst"])

array(['spam'], dtype=object)